## reading file
one thing needs to pay attention is that reading csv file will change list type to string. Json file will not have this problem 

In [7]:
import pandas as pd
import re
import json

train = pd.read_csv('combined_csv.csv')
train = train.drop(["Unnamed: 0"], axis=1)
train.head()

,Topic Title,Category,Tags,Author,Commenters,Leading Comment,Other Comments,Likes,Views
0,About the Buying/Selling category,Buying/Selling,[],system,[],Debating a new ride? Ask all your buying and s...,[],1,2.2k
1,New Car Salesperson Commissions,Buying/Selling,[],wolyrobb,"['wentwest', 'shadowfax', 'VOLVO-V70', 'Trieda...",I’ve been working with a particular salesperso...,['No one here can tell you what any dealers co...,12,559
2,Best used car to buy for long road trips?,Buying/Selling,['used-cars'],Whitey,"['texases', 'VOLVO-V70', 'Marnet', 'lion9car',...",Hello! I am new to the site and am having a di...,['The answer to your question depends far more...,44,14.6k
3,Honda Element any good?,Buying/Selling,"['honda', 'element']",texases,"['VOLVO-V70', 'old_mopar_guy', 'lockstar', 'sh...",Are these any good and were they discontinued?...,['A simple Google search would have shown that...,24,1.2k
4,1993 4Runner vs 2011 Mazda3 Hatch,Buying/Selling,"['toyota', '4runner', 'buying', 'used-cars']",galant,"['wolyrobb', 'Nevada_545', 'jordanethan609_175...",I’m conflicted between these two cars\n4Runner...,['You are comparing apples to hamsters with th...,46,1.7k


In [8]:
train =train[train.Category != ' 1990. My husband and I rented a spiffy new Cadillac in Indiana with plans to pick up my parents in Erie']

In [9]:
def replace_url(str):
    pattern =  r"/(http|https|ftp|ftps)\:\/\/[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(\/\S*)?/"
    result = re.sub(pattern," ", str)
    return result

train["Other Comments"] = train["Other Comments"].apply(lambda x: list(map(replace_url, x)) )
train.head()

,Topic Title,Category,Tags,Author,Commenters,Leading Comment,Other Comments,Likes,Views
0,About the Buying/Selling category,Buying/Selling,[],system,[],Debating a new ride? Ask all your buying and s...,"[[, ]]",1,2.2k
1,New Car Salesperson Commissions,Buying/Selling,[],wolyrobb,"['wentwest', 'shadowfax', 'VOLVO-V70', 'Trieda...",I’ve been working with a particular salesperso...,"[[, ', N, o, , o, n, e, , h, e, r, e, , c, ...",12,559
2,Best used car to buy for long road trips?,Buying/Selling,['used-cars'],Whitey,"['texases', 'VOLVO-V70', 'Marnet', 'lion9car',...",Hello! I am new to the site and am having a di...,"[[, ', T, h, e, , a, n, s, w, e, r, , t, o, ...",44,14.6k
3,Honda Element any good?,Buying/Selling,"['honda', 'element']",texases,"['VOLVO-V70', 'old_mopar_guy', 'lockstar', 'sh...",Are these any good and were they discontinued?...,"[[, ', A, , s, i, m, p, l, e, , G, o, o, g, ...",24,1.2k
4,1993 4Runner vs 2011 Mazda3 Hatch,Buying/Selling,"['toyota', '4runner', 'buying', 'used-cars']",galant,"['wolyrobb', 'Nevada_545', 'jordanethan609_175...",I’m conflicted between these two cars\n4Runner...,"[[, ', Y, o, u, , a, r, e, , c, o, m, p, a, ...",46,1.7k


In [10]:
train = train[train['Leading Comment'].notna()]

Clean string by changing url link into a space

In [11]:
words_analysis = pd.DataFrame()
comment_analysis = pd.DataFrame()


words_analysis['Leading Comment'] = train['Leading Comment']
words_analysis["lead_word_count"]= train['Leading Comment'].apply(lambda x: len(str(x).split(" ")))
comment_analysis['comment_count'] = len(train['Other Comments'])
comment_analysis['comment_word_count'] = list(map(lambda x: len(''.join(x).split(" ")),train['Other Comments']))


words_analysis.head()


,Leading Comment,lead_word_count
0,Debating a new ride? Ask all your buying and s...,12
1,I’ve been working with a particular salesperso...,152
2,Hello! I am new to the site and am having a di...,130
3,Are these any good and were they discontinued?...,192
4,I’m conflicted between these two cars\n4Runner...,183


In [12]:
words_analysis["lead_char_count"] = train['Leading Comment'].str.len()
comment_analysis["comment_char_count"] = train['Other Comments'].apply(lambda x:len(''.join(x)))
words_analysis.head()

,Leading Comment,lead_word_count,lead_char_count
0,Debating a new ride? Ask all your buying and s...,12,68
1,I’ve been working with a particular salesperso...,152,779
2,Hello! I am new to the site and am having a di...,130,648
3,Are these any good and were they discontinued?...,192,1000
4,I’m conflicted between these two cars\n4Runner...,183,953


## Finding stop words

necessary for the first time using nltk

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jianglu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [15]:
words_analysis["lead_stop_words"] = train['Leading Comment'].apply(lambda x: len([x for x in str(x).split() if x in stop]))
comment_analysis["comment_stop_words"] = train['Other Comments'].apply(lambda x: len([x for x in ''.join(x).split() if x in stop]))
words_analysis.head()

,Leading Comment,lead_word_count,lead_char_count,lead_stop_words
0,Debating a new ride? Ask all your buying and s...,12,68,4
1,I’ve been working with a particular salesperso...,152,779,57
2,Hello! I am new to the site and am having a di...,130,648,56
3,Are these any good and were they discontinued?...,192,1000,56
4,I’m conflicted between these two cars\n4Runner...,183,953,76


## preprocessing

to lowercase

In [16]:
train['Leading Comment'] = train['Leading Comment'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
train['Other Comments'] = train['Other Comments'].apply(lambda z: list(map(lambda x: " ".join(x.lower() for x in str(x).split()), z)))
train[['Leading Comment','Other Comments']]

,Leading Comment,Other Comments
0,debating a new ride? ask all your buying and s...,"[[, ]]"
1,i’ve been working with a particular salesperso...,"[[, ', n, o, , o, n, e, , h, e, r, e, , c, a, ..."
2,hello! i am new to the site and am having a di...,"[[, ', t, h, e, , a, n, s, w, e, r, , t, o, , ..."
3,are these any good and were they discontinued?...,"[[, ', a, , s, i, m, p, l, e, , g, o, o, g, l,..."
4,i’m conflicted between these two cars 4runner ...,"[[, ', y, o, u, , a, r, e, , c, o, m, p, a, r,..."
...,...,...
6603,i purchased a pre-owned 2004 lexus es330 95k m...,"[[, ', s, o, , y, o, u, , d, o, , h, a, v, e, ..."
6604,"i’m on my 3rd prius. i like it, but am thinkin...","[[, ', \, n, \, n, \, n, , r, e, d, g, l, a, m..."
6605,my 2009 hyundai santa fe won’t start. it start...,"[[, ]]"
6606,outside temp stays at 75f although it is over ...,"[[, ', i, , t, h, i, n, k, , t, h, i, s, , i, ..."


remove puctuation and stop words

In [17]:
# remove puc
train['Leading Comment'] = train['Leading Comment'].replace('[^\w\s]','')
train["Leading Comment"].head()

0    debating a new ride? ask all your buying and s...
1    i’ve been working with a particular salesperso...
2    hello! i am new to the site and am having a di...
3    are these any good and were they discontinued?...
4    i’m conflicted between these two cars 4runner ...
Name: Leading Comment, dtype: object

In [18]:
# remove stop words
train['Leading Comment'] = train['Leading Comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['Leading Comment'].head()

0    debating new ride? ask buying selling question...
1    i’ve working particular salesperson purchase n...
2    hello! new site dilemma… good used car buy dis...
3    good discontinued? still make 2021? would bett...
4    i’m conflicted two cars 4runner $4700 (193,000...
Name: Leading Comment, dtype: object

remove common word and rare word

In [19]:
# common word
freq1 = list(pd.Series(' '.join(train['Leading Comment']).split()).value_counts()[:10].index)
freq1

['car', 'would', 'fuel', 'gas', 'like', 'get', 'engine', 'one', 'new', 'i’m']

In [20]:
#rare word
freq2 = list(pd.Series(' '.join(train['Leading Comment']).split()).value_counts()[-10:].index)
freq2

['collapsing.',
 'axleback',
 'suing',
 '94,000',
 '(1986',
 'eye-popping',
 '“pete',
 'publicly',
 'hse',
 '15.8']

In [21]:
freq = freq1 + freq2
train['Leading Comment']  = train['Leading Comment'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['Leading Comment'].head()

0    debating ride? ask buying selling questions here.
1    i’ve working particular salesperson purchase c...
2    hello! site dilemma… good used buy distance dr...
3    good discontinued? still make 2021? better pic...
4    conflicted two cars 4runner $4700 (193,000) ma...
Name: Leading Comment, dtype: object

## Check spelling

In [22]:
from textblob import TextBlob
train['Leading Comment'][:5].apply(lambda x: str(TextBlob(x).correct()))

0     dealing ride? ask buying selling questions here.
1    i’ve working particular salesperson purchase c...
2    hello! site dilemma… good used buy distance dr...
3    good discontinued? still make 2021? better pic...
4    conflict two cars runner $4700 (193,000) madam...
Name: Leading Comment, dtype: object

In [23]:
#Tokenization
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/jianglu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
TextBlob(train['Leading Comment'][1]).words

WordList(['i', '’', 've', 'working', 'particular', 'salesperson', 'purchase', 'car', 'really', 'helpful', 'spent', 'time', 'select', 'correct', 'vehicle', 'ready', 'buy', 'i', '’', 've', 'tried', 'coming', 'dealership', 'couple', 'different', 'days', 'something', 'came', 'home', 'couldn', '’', 't', 'make', 'there', 'hard', 'find', 'time', 'afternoon', 'open', 'time', 'period', 'rare', 'could', 'go', 'make', 'purchase', 'salesperson', 'doesn', '’', 't', 'work', 'today', 'go', 'different', 'salesperson', 'making', 'sale', 'thinking', 'went', 'today', 'original', 'salesperson', 'who', 'put', 'time', 'lose', 'commission', '…', 'seems', 'disloyal', 'since', 'real', 'work', 'correct', 'today', '’', 's', 'salesperson', 'commission', 'thanks'])

In [25]:
#Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
train['Leading Comment'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0              debat ride? ask buy sell question here.
1    i’v work particular salesperson purchas car. r...
2    hello! site dilemma… good use buy distanc driv...
3    good discontinued? still make 2021? better pic...
4    conflict two car 4runner $4700 (193,000) mazda...
Name: Leading Comment, dtype: object

In [26]:
#Lemmatization
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/jianglu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
from textblob import Word
train['Leading Comment'] = train['Leading Comment'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train['Leading Comment'].head()

0     debating ride? ask buying selling question here.
1    i’ve working particular salesperson purchase c...
2    hello! site dilemma… good used buy distance dr...
3    good discontinued? still make 2021? better pic...
4    conflicted two car 4runner $4700 (193,000) maz...
Name: Leading Comment, dtype: object

 ## Advance Text Processing

In [28]:
# N-grams
TextBlob(train['Leading Comment'][1]).ngrams(2)

[WordList(['i', '’']),
 WordList(['’', 've']),
 WordList(['ve', 'working']),
 WordList(['working', 'particular']),
 WordList(['particular', 'salesperson']),
 WordList(['salesperson', 'purchase']),
 WordList(['purchase', 'car']),
 WordList(['car', 'really']),
 WordList(['really', 'helpful']),
 WordList(['helpful', 'spent']),
 WordList(['spent', 'time']),
 WordList(['time', 'select']),
 WordList(['select', 'correct']),
 WordList(['correct', 'vehicle']),
 WordList(['vehicle', 'ready']),
 WordList(['ready', 'buy']),
 WordList(['buy', 'i']),
 WordList(['i', '’']),
 WordList(['’', 've']),
 WordList(['ve', 'tried']),
 WordList(['tried', 'coming']),
 WordList(['coming', 'dealership']),
 WordList(['dealership', 'couple']),
 WordList(['couple', 'different']),
 WordList(['different', 'days']),
 WordList(['days', 'something']),
 WordList(['something', 'came']),
 WordList(['came', 'home']),
 WordList(['home', 'couldn']),
 WordList(['couldn', '’']),
 WordList(['’', 't']),
 WordList(['t', 'make']),
 

In [29]:
# Term frequency
tf1 = (train['Leading Comment'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,salesperson,5
1,i’ve,2
2,different,2
3,time,2
4,make,2
5,go,2
6,"today,",2
7,time.,1
8,hard,1
9,thinking,1


In [30]:
# Inverse Document Frequency
import numpy as np

for i,word in enumerate(tf1['words']):
      tf1.loc[i, 'idf'] = np.log(train.shape[0]/(len(train[train['Leading Comment'].str.contains(word)])))
tf1

/Users/jianglu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


error: unbalanced parenthesis at position 4

In [ ]:
# Term Frequency – Inverse Document Frequency (TF-IDF)
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
stop_words= 'english',ngram_range=(1,1))
train_vect = tfidf.fit_transform(train['Leading Comment'])
print(train_vect)

In [ ]:
words_analysis.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
stop_words= 'english',ngram_range=(1,1))
train_vect2 = tfidf.fit_transform(words_analysis['Leading Comment'])
print(train_vect2)

In [ ]:
tf2 = (words_analysis['Leading Comment'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf2.columns = ['words','tf']
tf2

In [ ]:
import numpy as np

for i,word in enumerate(tf2['words']):
      tf2.loc[i, 'idf'] = np.log(words_analysis.shape[0]/(len(words_analysis[words_analysis['Leading Comment'].str.contains(word)])))
tf2

In [ ]:
tf2['tfidf'] = tf2['tf'] * tf2['idf']
compare = pd.DataFrame({"words": tf1["words"],
                        "tf1_result": tf1['tfidf'],
                        "tf2_result": tf2['tfidf']})
compare

In [ ]:
words_analysis.isna().sum()

In [ ]:
train.isna().sum()

### Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(train['Leading Comment'])
print(train_bow)

In [ ]:
#Sentiment Analysis
train['Leading Comment'][:5].apply(lambda x: TextBlob(x).sentiment)
train['sentiment'] = train['Leading Comment'].apply(lambda x: TextBlob(x).sentiment[0] )
words_analysis['sentiment'] = train['sentiment']
train[['Leading Comment','sentiment']].head()

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

In [ ]:
from gensim.models import KeyedVectors # load the Stanford GloVe model
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

## Exploratory Data Analysis for Natural Language Processing: A Complete Guide to Python Tools

In [ ]:
corpus=[]
new= train['Leading Comment'].str.split()
new=new.values.tolist()
corpus=[word for i in new for word in i]

from collections import defaultdict
dic=defaultdict(int)
for word in corpus:
    if word in stop:
        dic[word]+=1

In [ ]:
def get_top_ngram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) 
                  for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:10]

In [ ]:
import seaborn as sns

top_n_bigrams=get_top_ngram(train['Leading Comment'],2)[:10]
x,y=map(list,zip(*top_n_bigrams))
sns.barplot(x=y,y=x)

In [ ]:
top_tri_grams=get_top_ngram(train['Leading Comment'],n=3)
x,y=map(list,zip(*top_tri_grams))
sns.barplot(x=y,y=x)

## Topic Modeling exploration with pyLDAvis

In [ ]:
import sys
print(sys.executable)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

def show_wordcloud(data):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=100,
        max_font_size=30,
        scale=3,
        random_state=1)
   
    wordcloud=wordcloud.generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(corpus)